# Arquivo para treinamento do Yolo para reconhecer raças de cachorros

dataset: https://www.kaggle.com/jessicali9530/stanford-dogs-dataset

### Preparando o ambiente:

In [2]:
# O dataset inicialmente é composto 
import os
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

In [3]:
def list_files(directory, extension):
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith(extension):
                files.append(os.path.relpath(os.path.join(root, filename), directory))
    return files

def convert_xml_to_yolo(xml_file, classes):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    yolo_annotations = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = classes.index(class_name)

        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        x_center = (xmin + xmax) / 2.0 / width
        y_center = (ymin + ymax) / 2.0 / height
        bbox_width = (xmax - xmin) / float(width)
        bbox_height = (ymax - ymin) / float(height)

        yolo_annotations.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

    return yolo_annotations


Na célula abaixo, selecione o caminho para o dataset e as raças que deseja treinar o modelo, copie e cole os nomes das raças desejadas no array `dataset_folders`. Os nomes se repetem no image folder e no annotation folder.

No exemplo abaixo foram selecionadas 10 raças de cachorros.

In [ ]:
dataset_dir = 'data'

# Selecione as classes que deseja utilizar
dataset_folders = ["n02085620-Chihuahua", "n02086240-Shih-Tzu", "n02094433-Yorkshire_terrier", "n02099601-golden_retriever", "n02106166-Border_collie", "n02106550-Rottweiler", "n02107142-Doberman", "n02108089-boxer", "n02109047-Great_Dane", "n02110958-pug"]
dog_names = [nome.split("-",1)[1] for nome in dataset_folders]
print(dog_names)

In [ ]:
if os.path.exists(os.path.join(dataset_dir, 'Annotation')):
    shutil.move(os.path.join(dataset_dir, 'Annotation'), os.path.join(dataset_dir, 'labels'))
images_dir = os.path.join(dataset_dir, 'images')
print(images_dir)
label_dir = os.path.join(dataset_dir, 'labels')
print(label_dir)

In [19]:
# Percorre o dataset e deleta as pastas que não estão na lista de classes
for root, dirs, files in os.walk(images_dir):
    for dir in dirs:
        if not dir in dataset_folders:
            shutil.rmtree(os.path.join(images_dir, dir))
            shutil.rmtree(os.path.join(label_dir, dir))

In [ ]:
# Converter anotações XML para .txt, no formato YOLO
for root, _, files in os.walk(label_dir):
    for file in files:
        xml_path = os.path.join(root, file)
        
        # Verificar se o arquivo é XML
        try:
            tree = ET.parse(xml_path)
            print(f"Arquivo XML encontrado: {xml_path}")
            yolo_annotations = convert_xml_to_yolo(xml_path, dog_names)

            # Salvar anotações YOLO em arquivo .txt correspondente
            yolo_path = os.path.splitext(xml_path)[0] + '.txt'
            print(f"Salvando anotações YOLO em: {yolo_path}")
            with open(yolo_path, 'w') as f:
                f.write('\n'.join(yolo_annotations))
        except ET.ParseError:
            print(f"Arquivo não é um XML válido: {xml_path}")

print("Conversão das anotações concluída.")

In [ ]:
# Listar todas as imagens e anotações
images = list_files(images_dir, '.jpg')
print(images)
annotations = list_files(label_dir, '.txt')
print(annotations)

# Dividir em treino e validação (80% treino, 20% validação)
train_images, val_images, train_annotations, val_annotations = train_test_split(
    images, annotations, test_size=0.2, random_state=42
)

# Cria o diretório de treino e validação
train_dir = os.path.join(dataset_dir, 'train')
os.makedirs(train_dir, exist_ok=True)

validation_dir = os.path.join(dataset_dir, 'val')
os.makedirs(validation_dir, exist_ok=True)

# Adiciona as pastas de imagens e anotações
train_images_dir = os.path.join(train_dir, 'images')
os.makedirs(train_images_dir, exist_ok=True)

train_label_dir = os.path.join(train_dir, 'labels')
os.makedirs(train_label_dir, exist_ok=True)

val_images_dir = os.path.join(validation_dir, 'images')
os.makedirs(val_images_dir, exist_ok=True)

val_label_dir = os.path.join(validation_dir, 'labels')
os.makedirs(val_label_dir, exist_ok=True)

# Coloca os arquivos dentro das pastas de treino e validação
for image, annotation in zip(train_images, train_annotations):
    image_path = os.path.join(images_dir, image)
    annotation_path = os.path.join(label_dir, annotation)
    shutil.move(image_path, train_images_dir)
    shutil.move(annotation_path, train_label_dir)

for image, annotation in zip(val_images, val_annotations):
    image_path = os.path.join(images_dir, image)
    annotation_path = os.path.join(label_dir, annotation)
    shutil.move(image_path, val_images_dir)
    shutil.move(annotation_path, val_label_dir)

#Deleta as pastas images e labels
shutil.rmtree(images_dir)
shutil.rmtree(label_dir)

Por fim, vamos adicionar um arquivo .yaml na raiz do dataset com o seguinte conteúdo:

In [25]:
# Cria o arquivo data.yaml
data_yaml = f"""
train: {os.path.abspath(train_images_dir)}
val: {os.path.abspath(val_images_dir)}
nc: {len(dog_names)}
names: {dog_names}
"""

with open(os.path.join(dataset_dir, 'data.yaml'), 'w') as f:
    f.write(data_yaml)

Esse arquivo será utilizado para treinar o modelo.

**Pronto!** Seu dataset está pronto para ser treinado. 

---

## Treinamento do Modelo:

Foi utilizado um modelo pré-treinado do YoloV11.
O processo abaixo pode demorar de acordo com a quantidade de raças escolhidas, bem como a capacidade de processamento da máquina.

Para baixar um modelo pré-treinado, acesse o link:	
https://docs.ultralytics.com/tasks/detect/#models

Garanta que o ambiente de execução está configurado para GPU.

O comando abaixo informa, para placas de vídeo NVIDIA, qual a versão do CUDA está instalada.

In [ ]:
!nvidia-smi

Garanta que o pytorch está instalado e configurado para GPU.

Caso o comando abaixo retorne **False**, instale o pytorch com suporte a versão do CUDA utilizado pela sua GPU.

O download pode ser feito no link: https://pytorch.org/get-started/locally/

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


Os parâmetros de treinamento podem ser ajustados de acordo com a necessidade.

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt') # Insira o caminho para o modelo pré-treinado

# testa se o CUDA está disponível
if __name__ == '__main__':

    results = model.train(
        data='C:/Users/luis/Documents/pessoal/yoloProject/data/data.yaml',  # Caminho para o arquivo data.yaml que criamos
        epochs=50,                                                          # Número de épocas de treino (ajuste conforme necessário)
        imgsz=640,                                                          # Tamanho da imagem de entrada
        batch=16,                                                           # Tamanho do batch (ajuste conforme memória disponível)
        workers=4                                                           # Número de workers para carregamento dos dados
    )

    # Salva o modelo treinado
    model.save('yolo11n.pt')
    

Após o treinamento, o modelo será salvo no próprio arquivo yolo11n.pt, que pode ser utilizado para inferência.